# Amazon SageMaker の Factorization Machines と BlazingText を使用したレコメンダ システムの構築

---

---

## 背景

- Factorization Machines とは、因数分解機です。（以下、Factorization Machines）
- レコメンダ システムは、アマゾン、並びにネットフリックス Prizeの例からもみれるように、機械学習のカタリストとなりました。
  - ネットフリックス Prizeとは：2006年から2009年にかけて、三年間にわたって行なわれた史上最大級のアルゴリズム・コンテスト。賞金100万ドルを賭けて，186ヶ国から4万チームが参戦して争われた壮大なアルゴリズム競争です。
- ユーザー・アイテムの Matrix Factorization　（又は、Matrix Decompositionと言い、日本語では行列の分解） は、コア・中核的な手法です。
  - 行列の分解とは：行列と行列の積への分解。
  - 何の為に分解するのか？行列を分解することで、計算を速く行えるようになる、という実際的なメリットがあったり、その行列の性質がわかったりするからです。
- Factorization Machines は、linear prediction（線形予測） とペアとなったフィーチャの相互作用が因数分解された表現を組み合わせます。

$$\hat{r} = w_0 + \sum_{i} {w_i x_i} + \sum_{i} {\sum_{j > i} {\langle v_i, v_j \rangle x_i x_j}}$$

- Amazon SageMaker の built-in Factorization Machines は高度にスケーラブルです。

---

## セットアップ

以下のセルを実行する前に：
1. マネージメントコンソールにて、SageMaker にてノートブックインスタンスを立ち上げて下さい。
2. 作成時に、SageMaker の IAM ポリシーをそのノートブックインスタンスに追加して、S3 の　read/write　アクセスを許可するように設定します。

セル１と２は：
3. S3 バケット、セッション等の作成。(first code cell)
4. 必要なライブラリのインポート (second code cell)

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
base = 'DEMO-loft-recommender'
prefix = 'sagemaker/' + base

role = sagemaker.get_execution_role()

In [ ]:
import sagemaker
import os
import pandas as pd
import numpy as np
import boto3
import json
import io
import matplotlib.pyplot as plt
import sagemaker.amazon.common as smac
from sagemaker.predictor import json_deserializer
from scipy.sparse import csr_matrix

---

## データ

[Amazon Reviews AWS Public Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)
Amazonカスタマーレビュー（商品レビュー）は、Amazonの象徴的な商品の1つです。 1995 年の最初のレビューから20 年以上の期間、何百万人ものお客様が、Amazon.com ウェブサイト上の商品に関する意見を述べ経験をシェアするために、1 億以上のレビューを投稿してきました。 これにより、Amazonカスタマーレビューは、自然言語処理（NLP）、情報検索（IR）、機械学習（ML）などの分野の学術研究者のための豊富な情報源となっています。 これに伴い、お客様の製品体験の理解に関する複数の分野におけるさらなる研究を目的として、このデータを公開しています。 具体的には、このデータセットは、顧客の評価と意見のサンプル、地理的・地域全体にわたる製品の認識の変動、およびレビューにおける販促意図またはバイアスを表すために構築されました。

その内容は、
- 1 から 5 つ星評価。
- 2百万以上の お客様によるレビュー。
- このノートブックでは、16万以上の デジタルビデオのレビューを使用して、機械学習を行います。 

以下のセルを実行し、データセットをノートブックインスタンスの /tmp/recsys/ にダウンロードします。

In [ ]:
!mkdir /tmp/recsys/
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz /tmp/recsys/

pandas の read_csv() を使い、メモリ内の dataframe にデータセットをロードします。

In [ ]:
df = pd.read_csv('/tmp/recsys/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

データセットの列は下記の通りです:

- `marketplace`: 2 文字の国コード (このデータセットは、全てUS,「米国」となっています）。
- `customer_id`: ランダムに割り当てたお客様番号又はID。 
- `review_id`: レビューをユニークに識別できるID。
- `product_id`: Amazon 標準識別番号 (ASIN)。  
- `product_parent`: そのASINの親。 複数のASIN（同じ商品のカラーバリエーションまたはフォーマットのバリエーション）を１つの親にまとめることができます。
- `product_title`: 商品のタイトル。
- `product_category`: グループレビューに使用できる幅広い製品カテゴリ（このデータセットの場合はデジタルビデオ）
- `star_rating`: レビューの評価 (1 から 5 つ星)。
- `helpful_votes`: レビューが役立つと投票された数。
- `total_votes`: レビューが受け取った投票総数。
- `vine`: レビューは [Vine](https://www.amazon.com/gp/vine/help) のプログラムの一部として書かれているか否か。
- `verified_purchase`: 確認済みの購入からのレビューか否か。
- `review_headline`: レビューのタイトル。
- `review_body`: レビューのテキスト。
- `review_date`: レビューが書き込まれた日付。

以下のセルで、トレーニングに使用するフィールド・列だけを選び、使用しない列を削除します。

In [ ]:
df = df[['customer_id', 'product_id', 'product_title', 'star_rating', 'review_date']]

In [ ]:
df.to_csv(r'reviews.csv') #メモリに存在するデータフレームをcsvに書き出し保存します。このcsvは次のハンズオンで使います。

In [ ]:
df.head()

以下のセルを実行すると、ほとんどのユーザーは、ほとんどの映画を評価しないことが分かります。 

例えば、customersの統計を見てみますと、50パーセンタイルまでカウントが１、すなわちユーザは1つの評価しか投稿していないということになります。
2万のユニークなお客様の中でおおよそ半分は投稿数が１つということです。

In [ ]:
customers = df['customer_id'].value_counts() #customer_idを数え、その値を返します。
products = df['product_id'].value_counts()   #product_idを数え、その値を返します。

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1]
print('customers count\n', customers.size)
print('customers\n', customers.quantile(quantiles)) #上記のvalue_counts()で返された値を、統計してみてみます。
print('products count \n', products.size)
print('products\n', products.quantile(quantiles))

以下のセルで映画を多く評価していないお客様と、評価を多数受けてない映画を除外します。

それを、5つ以上投稿したお客様と、10件以上のレビューを受けた映画だけをキープすることで実装します。

それにより、約14万程のお客様に絞り込みました。映画の数は約3万８千となりました。（以下、customer_index と product_index をセルにて実行することにより確認できます。

そして、その customerId と productId を reduced_df としてメモリにセーブします。

In [ ]:
customers = customers[customers >= 5]
products = products[products >= 10]

reduced_df = df.merge(pd.DataFrame({'customer_id': customers.index})).merge(pd.DataFrame({'product_id': products.index}))

お客様と映画の連番インデックス（sequential index）を作成し user と item と呼びます。

In [ ]:
customers = reduced_df['customer_id'].value_counts()
products = reduced_df['product_id'].value_counts()

In [ ]:
customer_index = pd.DataFrame({'customer_id': customers.index, 'user': np.arange(customers.shape[0])})
product_index = pd.DataFrame({'product_id': products.index, 
                              'item': np.arange(products.shape[0]) + customer_index.shape[0]})

reduced_df = reduced_df.merge(customer_index).merge(product_index)
reduced_df.head()

最初のレビューからの日数を数え、days_since_firstを作成する（トレンドをキャプチャするfeatureとして使えます）

In [ ]:
reduced_df['review_date'] = pd.to_datetime(reduced_df['review_date'])
customer_first_date = reduced_df.groupby('customer_id')['review_date'].min().reset_index()
customer_first_date.columns = ['customer_id', 'first_review_date']

In [ ]:
reduced_df = reduced_df.merge(customer_first_date)
reduced_df['days_since_first'] = (reduced_df['review_date'] - reduced_df['first_review_date']).dt.days
reduced_df['days_since_first'] = reduced_df['days_since_first'].fillna(0)

In [ ]:
reduced_df

以下のセルにて、学習用のデータととテスト用のデータに分けます。

In [ ]:
test_df = reduced_df.groupby('customer_id').last().reset_index()

train_df = reduced_df.merge(test_df[['customer_id', 'product_id']], 
                            on=['customer_id', 'product_id'], 
                            how='outer', 
                            indicator=True)
train_df = train_df[(train_df['_merge'] == 'left_only')]

- Factorization Machinesは、以下のデータをインプットとして受け取ります。
  - Sparse matrix（疎行列又は、スパース行列）。ほとんど0で構成されている行列。
  - ターゲット変数は、映画に対するそのお客様（ユーザー）の評価です。
  - ユーザーのワンホットエンコーディング ($N$ features)　この場合、$N$は140344。
  - 映画（アイテム）のワンホットエンコーディング ($M$ features)　$M$は38385。

|Rating|User1|User2|...|UserN|Movie1|Movie2|Movie3|...|MovieM|Feature1|Feature2|...|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|4|1|0|...|0|1|0|0|...|0|20|2.2|...|
|5|1|0|...|0|0|1|0|...|0|17|9.1|...|
|3|0|1|...|0|1|0|0|...|0|3|11.0|...|
|4|0|1|...|0|0|0|1|...|0|15|6.4|...|


- 完全な行列をメモリに保持したくありません。従って..
  - 疎行列を作成する。
  - 疎行列は、CPUで効率的に動作するように設計されています。 より密度の高い行列のトレーニングの一部は、GPUで並列化することができます。
  
以下の function で、drameframe から、scipy の csr_matrix に変換します。


In [ ]:
def to_csr_matrix(df, num_users, num_items):
    feature_dim = num_users + num_items + 1
    data = np.concatenate([np.array([1] * df.shape[0]),
                           np.array([1] * df.shape[0]),
                           df['days_since_first'].values])
    row = np.concatenate([np.arange(df.shape[0])] * 3)
    col = np.concatenate([df['user'].values,
                          df['item'].values,
                          np.array([feature_dim - 1] * df.shape[0])])
    return csr_matrix((data, (row, col)), 
                      shape=(df.shape[0], feature_dim), 
                      dtype=np.float32)

In [ ]:
train_csr = to_csr_matrix(train_df, customer_index.shape[0], product_index.shape[0])
test_csr = to_csr_matrix(test_df, customer_index.shape[0], product_index.shape[0])

以下、SageMakerのfactorization machinesが必要とするスパース・レコードIOに包まれたprotobufに変換します。そして、S3バケットにアップロードします。

In [ ]:
def to_s3_protobuf(csr, label, bucket, prefix, channel='train', splits=10):
    indices = np.array_split(np.arange(csr.shape[0]), splits)
    for i in range(len(indices)):
        index = indices[i]
        buf = io.BytesIO()
        smac.write_spmatrix_to_sparse_tensor(buf, csr[index, ], label[index])
        buf.seek(0)
        boto3.client('s3').upload_fileobj(buf, bucket, '{}/{}/data-{}'.format(prefix, channel, i))

In [ ]:
to_s3_protobuf(train_csr, train_df['star_rating'].values.astype(np.float32), bucket, prefix)
to_s3_protobuf(test_csr, test_df['star_rating'].values.astype(np.float32), bucket, prefix, channel='test', splits=1)

---

## 学習

- トレーニング・ジョブを実行するための [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) estimatorを作成し、以下の項目を指定します。
  - アルゴリズムが保存されているコンテナのイメージ
  - IAM ロール
  - ハードウェアのセットアップ
  - アウトプットを保存する S3 のバケット
  - アルゴリズムのハイパーパラメータ
    - `feature_dim`: $N + M + 1$ (追加の feature は、トレントをキャプチャする `days_since_first`)
    - `num_factors`: 因数分解された交互作用で減少させた dimension
    - `epochs`: データセットを通過させる回数
- `.fit()` は S3 の学習用とテスト用データを指定し、トレーニングジョブを開始します。
- train_instance_count=4 とあるように分散学習の例となります。
- この例では、cタイプのインスタンスを利用しています。
- また、トレーニング終了後、このインスタンスは直ちに処分され、コスト軽減となります。

In [ ]:
fm = sagemaker.estimator.Estimator(
    sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, 'factorization-machines', 'latest'),
    role, 
    train_instance_count=4, 
    train_instance_type='ml.c5.2xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    base_job_name=base,
    sagemaker_session=sess)

fm.set_hyperparameters(
    feature_dim=customer_index.shape[0] + product_index.shape[0] + 1,
    predictor_type='regressor',
    mini_batch_size=1000,
    num_factors=256,
    epochs=3)

fm.fit({'train': sagemaker.s3_input('s3://{}/{}/train/'.format(bucket, prefix), distribution='ShardedByS3Key'), 
        'test': sagemaker.s3_input('s3://{}/{}/test/'.format(bucket, prefix), distribution='FullyReplicated')})

---

## ホスト・デプロイ

学習を終えたモデルを、リアルタイムの本番環境へエンドポイントとしてデプロイします。
- この際、mタイプのインスタンスを使用します。
- initial_instance_count=1 とありますが、エンドポイント利用可能後、APIへのリクエストが増えると自動的にスケールアウトします。


In [ ]:
fm_predictor = fm.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

API呼び出しの際に使う、メモリ内のデータをシリアル化する必要があります。predictor にてシリアライザを指定します。　

In [ ]:
def fm_serializer(df):
    feature_dim = customer_index.shape[0] + product_index.shape[0] + 1
    js = {'instances': []}
    for index, data in df.iterrows():
        js['instances'].append({'data': {'features': {'values': [1, 1, data['days_since_first']],
                                                      'keys': [data['user'], data['item'], feature_dim - 1],
                                                      'shape': [feature_dim]}}})
    return json.dumps(js)

In [ ]:
fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

単一のユーザー・アイテムのリアルタイム予測は、以下の用に行います。

In [ ]:
test_df.head(1)

In [ ]:
fm_predictor.predict(test_df.head(1))

エンドポイントを削除します。

In [ ]:
fm_predictor.delete_endpoint()

---

---

# Extra credit

- 新しい映画を追加するとどうなるのでしょうか？
  - データセット内に「1」と設定するフィーチャがない。
  - 類似商品を見つけるための過去の評価がない。
  - Factorization Machines において、コールドスタートの問題は難解です。
- そこで、Word2vecを使用。
  - Word embeddings を使い、自然言語処理を行う。類似の単語は同様のベクトルとなる。
  - 連結された商品タイトルを、カスタマーレビュー履歴の文章として使用する。
  - SageMaker の BlazingText はサブワードを扱うことができ、非常に早く実装することを可能にします。

---

## データ

まず、商品タイトルを連結して、それぞれを 1 つの単語として扱います。

In [ ]:
reduced_df['product_title'] = reduced_df['product_title'].apply(lambda x: x.lower().replace(' ', '-'))

そして、お客様の購入履歴の書き込みを行う。

In [ ]:
first = True
with open('customer_purchases.txt', 'w') as f:
    for customer, data in reduced_df.sort_values(['customer_id', 'review_date']).groupby('customer_id'):
        if first:
            first = False
        else:
            f.write('\n')
        f.write(' '.join(data['product_title'].tolist()))

In [ ]:
reduced_df.head()

SageMakerの学習が、データを使用できるようにS3に書き込む。

In [ ]:
inputs = sess.upload_data('customer_purchases.txt', bucket, '{}/word2vec/train'.format(prefix))

---

## 学習

以下のセルで、SageMaker のestimatorを作成する:
- トレーニングジョブの引数を指定する。その設定のポイントは：
  - blazingtext のコンテナを指定。
  - P3 インスタンスを使用。
  - アウトプットは S3 指定したバケットに保存。
- ハイパーパラメータを設定する。その設定のポイントは：
  - 5 回未満登場るすタイトルを削除する。
  - 100 dimensionalのサブスペースに埋め込む。Embed in a 100-dimensional subspace
  - サブワードを使用してタイトルの類似性を取り込む。Use subwords to capture similarity in titles
fit() を実行し、学習を開始する。

In [ ]:
bt = sagemaker.estimator.Estimator(
    sagemaker.amazon.amazon_estimator.get_image_uri(boto3.Session().region_name, 'blazingtext', 'latest'),
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    train_volume_size = 5,
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sess)

bt.set_hyperparameters(mode="skipgram",
    epochs=10,
    min_count=5,
    sampling_threshold=0.0001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    min_char=5,
    max_char=10,
    evaluation=False,
    subwords=True)

bt.fit({'train': sagemaker.s3_input(inputs, distribution='FullyReplicated', content_type='text/plain')})

---

## モデル

- S3 に保存されたモデルを抽出する。
- そして実際、embeddingsを見てみましょう。

In [ ]:
!aws s3 cp $bt.model_data ./

In [ ]:
!tar -xvzf model.tar.gz

Pandas の read_csv() を使い vectors.txt をメモリにロードします。

In [ ]:
vectors = pd.read_csv('vectors.txt', delimiter=' ', skiprows=2, header=None)

Embeddings 自体の数値をみても分かりにくいことがうかがえます。

In [ ]:
vectors.sort_values(1)

In [ ]:
vectors.sort_values(2)

１００個のdimentionがあることが伺えます。 これをt-SNEを使いさらにdimentionの数を減らし、上位 100タイトルをマッピングしてみましょう。Yes, but there's 100.  Let's reduce this further with t-SNE and map the top 100 titles.

In [ ]:
product_titles = vectors[0]
vectors = vectors.drop([0, 101], axis=1)

**t-SNEとは**

t 分散確率近傍埋め込み（t-SNE）は、高次元データセットのビジュアル化に特に適した次元縮小のための技術です。

データ点間の類似性を共同確率に変換し、低次元埋め込みの共同確率と高次元データの間のKullback-Leiblerの発散を最小限に抑えようとします。t-SNEは凸状ではないコスト関数を持っています。つまり、異なる初期化で 異なる結果がでます。

以下で、sklearn の実装した t-SNE を使用します。

t-Distributed Stochastic Neighbor Embedding (t-SNE) is a (prize-winning) technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets.

It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=10000)
embeddings = tsne.fit_transform(vectors.values[:100, ])

In [ ]:
from matplotlib import pylab
%matplotlib inline

def plot(embeddings, labels):
    pylab.figure(figsize=(20,20))
    for i, label in enumerate(labels):
        x, y = embeddings[i,:]
        pylab.scatter(x, y)
        pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                       ha='right', va='bottom')
    pylab.show()

plot(embeddings, product_titles[:100])

---

## ホスト・デプロイ

モデルをリアルタイムで対応できるエンドポイントとしてデプロイします。

In [ ]:
bt_endpoint = bt.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

以下のセルで、推論に使うタイトルを指定します。（一部は実際に存在するタイトルで、一部は作成されたタイトルです）。
そのタイトルを使い、アウトプットを比較します。

In [ ]:
words = ["sherlock-season-1", 
         "sherlock-season-2",
         "sherlock-season-5",
         'arbitrary-sherlock-holmes-string',
         'the-imitation-game',
         "abcdefghijklmn",
         "keeping-up-with-the-kardashians-season-1"]

payload = {"instances" : words}

response = bt_endpoint.predict(json.dumps(payload))

vecs_df = pd.DataFrame(json.loads(response))

相関と距離を計算します。Calculate correlation and distance.

In [ ]:
vecs_df = pd.DataFrame(vecs_df['vector'].values.tolist(), index=vecs_df['word'])

In [ ]:
vecs_df = vecs_df.transpose()
vecs_df.corr()

In [ ]:
for column in vecs_df.columns:
    print(column + ':', np.sum((vecs_df[column] - vecs_df['sherlock-season-1']) ** 2))

'sherlock-season-1'と関連しているのは:
- 'sherlock-season-5' は実際存在しない、作られたタイトルですが、'sherlock-season-2と共に'sherlock-season-1'とよく関連しています。
- 'arbitrary-sherlock-holmes-string' は、また作られたタイトルですが、それ程ではないですが関連はまだ強い。is also made up and relates less well but still fairly strong
- 'the-imitation-game' はベネディクト・カンバーバッチ主演の人気のあるプライムビデオタイトルであり、中くらいの関係を持っています。任意のシャーロックのタイトルよりも関連は薄いです。is another popular Prime video title starring Benedict Cumberbatch and has a moderate relationship, but worse than the arbitrary Sherlock title
- 'abcdefghijklmn' は作られたタイトルで、さらに関連は薄い。
- 'keeping-up-with-the-kardashians-season-1'はさらに関係が薄いとでる。 

最後に、エンドポイントを削除します。

In [ ]:
bt_endpoint.delete_endpoint()

---
##  ハンズオン終了時に、ノートブックインスタンスを停止して下さい! 
---

# 最後に

- Factorization Machinesは、大規模なデータセットに対するレコメンダ システムを迅速かつ正確に構築することを可能にします。
- やってみること：
  - 拡張する為に、フィーチャを追加することを試してみる。　
  - Factorization Machines以外の他の方法と比較してみる。
  - 2つのモデルをアンサンブルとして使ってみる